# Análine de dados do Thermal Shift

Desenvolvido por **Vitória Yumi Uetuki Nicoleti**.

Observações iniciais:

Tenha certeza de que o arquivo Excel que contém os dados que quer analisar está na mesma pasta que este '.ipynb' em questão. 

**Importação das bibliotecas que são necessárias para o código**

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from lmfit.models import StepModel
import math
import seaborn as sns
plt.style.use("seaborn")

**Definições das funções**

In [ ]:
def encontra_maximo(lista):
    
    """ Encontra valor máximo em uma lista """
    
    valor = lista[0]
    for n in lista:
        if n > valor:
            valor = n
    return valor

def encontra_minimo(lista):
    
    """ Encontra valor mínimo em uma lista """
    
    valor = lista[0]
    for n in lista:
        if n < valor:
            valor = n
    return valor

def derivada_xy(x_vals, y_vals):
    
    """ Calcula a derivada, dados valores de x e y, retornando uma lista com as médias entre dois pontos feitas e a outra com as derivadas """
    
    avg_x_vals = []
    deriv_vals = []
    for i in range(len(x_vals) - 1):
        deriv = ( y_vals[i + 1] - y_vals[i] ) / ( x_vals[i + 1] - x_vals[i] )
        avg_x = ( x_vals[i + 1] + x_vals[i] ) / 2

        avg_x_vals.append(avg_x)
        deriv_vals.append(deriv)

    return avg_x_vals, deriv_vals 

def sigmoide(x, A, u, o):
    
    """ Define a formula de sigmóide para poder dar parâmetros e o valor de x e encontrar o y correspondente
    Nela: 
    A -> amplitude
    u -> centro
    o -> sigma
    a -> alfa
    x -> valor de x """
    
    a = (x-u)/o
    f = A*(1-(1/(1 + e ** (a))))
    
    return f

def getIndexes(dfObj, value):
    
    """ Encontra index de um elemento em DataFrame """
     
    # Empty list
    listOfPos = []
     
    # isin() method will return a dataframe with
    # boolean values, True at the positions   
    # where element exists
    result = dfObj.isin([value])
     
    # any() method will return
    # a boolean series
    seriesObj = result.any()
 
    # Get list of column names where
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            listOfPos.append((row, col))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return listOfPos

## Definindo número de euler com numpy
e = np.e

## Estabelecendo listas necessárias
Tms = []
Observacoes = []
CORTE_C = []
CORTE_T = []
A_MIN = []
A_MAX = []
A_ESTIMADO = []
C_MIN = []
C_MAX = []
C_ESTIMADO = []
OBS_MODEL = []
AMPLITUDE = []
CENTRO = []
SIGMA = []
PONTO_INFLEXAO = []
LEITURA_PI = []
POSICAO_PI = []

def tratamento_dados_poco():
    
    """ Essa função é referente a todo o processo de tratamento de dados de um poço, desde sua escolha até o retorno de sua Tm e plot.
        Recolhe os dados em listas. """
   
    ## Escolha de poço
    print("Qual o poço que quer plotar? (exemplo: D4)")
    POCO_PLOTAR_i = input()
    POCO_PLOTAR = pocos.index(POCO_PLOTAR_i)
    
    ## Plotagem dos seus dados
    df_poco_plotar = dfs[POCO_PLOTAR]
    df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
    plt.title(POCO_PLOTAR_i)
    plt.show()
    
    ## Observação inicial
    print("Caso o gráfico já indique a desnaturação da proteína (não dê para modelar sigmóide, só decresce), responda 'Desnaturou'; Caso dê para modelar, responda 'Modelar'")
    OBS = input()
    Observacoes.append(OBS)
    
    while OBS.lower() not in ["desnaturou", "modelar"]:
        print("Escreva conforme o indicado")
        OBS = input()
    
    if OBS.lower() == 'desnaturou':
        Tm = 0
        Tms.append(Tm)
        OBS_MODEL.append(OBS)
        CORTE_C.append(0)
        CORTE_T.append(0)
        A_MIN.append(0)
        A_MAX.append(0)
        A_ESTIMADO.append(0)
        C_MIN.append(0)
        C_MAX.append(0)
        C_ESTIMADO.append(0)
        AMPLITUDE.append(0)
        CENTRO.append(0)
        SIGMA.append(0)
        PONTO_INFLEXAO.append(0)
        LEITURA_PI.append(0)
        POSICAO_PI.append(0)
        resposta = print("A proteína já estava desnaturada nessas condições desde o início. Logo, atribuiu-se Tm igual a ", Tm)
          
        return resposta
    
    elif OBS.lower() == 'modelar':
    
        ## Perguntas sobre corte
        print("Considerar dados a partir de qual valor de Reading do gráfico?")
        CORTE_c = int(input())
        print("Considerar dados até qual valor de Reading do gráfico?")
        CORTE_t = int(input())

        ## Corte
        logic_c = df_poco_plotar["Reading"] >= CORTE_c
        df_poco_plotar = df_poco_plotar.loc[logic_c]
        logic_t = df_poco_plotar["Reading"] <= CORTE_t
        df_poco_plotar = df_poco_plotar.loc[logic_t]

        ## Plotagem da edição
        print("Após corte:")
        df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
        plt.title(POCO_PLOTAR_i)
        plt.show()
        
        ## Conferindo
        print("Caso queira refazer o corte, responda 'Refazer'; Caso já esteja satisfeito(a), responda 'Prosseguir'")
        CORTE = input()
        
        while CORTE.lower() not in ["refazer", "prosseguir"]:
            print("Escreva conforme o indicado")
            CORTE = input()
            
        while CORTE.lower() == 'refazer':
            print("Considerar dados a partir de qual valor de Reading do gráfico?")
            CORTE_c = int(input())
            print("Considerar dados até qual valor de Reading do gráfico?")
            CORTE_t = int(input())
            
            df_poco_plotar = dfs[POCO_PLOTAR]
            
            ## Corte
            logic_c = df_poco_plotar["Reading"] >= CORTE_c
            df_poco_plotar = df_poco_plotar.loc[logic_c]
            logic_t = df_poco_plotar["Reading"] <= CORTE_t
            df_poco_plotar = df_poco_plotar.loc[logic_t]

            ## Plotagem da edição
            print("Após corte:")
            df_poco_plotar.plot("Reading", "Fluorescence", linestyle="", marker="o")
            plt.title(POCO_PLOTAR_i)
            plt.show()
            
            print("Caso queira refazer o corte, responda 'Refazer'; Caso já esteja satisfeito(a), responda 'Prosseguir'")
            CORTE = input()

        ## Normalização
        eixo_y = list(df_poco_plotar["Fluorescence"])
        valor_max = encontra_maximo(eixo_y)
        valor_min = encontra_minimo(eixo_y)
        Fluorescence_normalized = []
        for n in eixo_y:
            novo_valor = (n - valor_min)/(valor_max - valor_min) * 100
            Fluorescence_normalized.append(novo_valor)
        df_poco_plotar.insert(4, "Fluorescence_normalized", Fluorescence_normalized, allow_duplicates=False)
        print("Dataframe do poço ", POCO_PLOTAR_i, " com normalização:")
        display(df_poco_plotar)

        ## Plotagem da normalização
        print("Após normalização:")
        plt.scatter(df_poco_plotar["Reading"], df_poco_plotar["Fluorescence_normalized"])
        plt.title(POCO_PLOTAR_i)
        plt.show()

        ## Implementação do modelo
        modelo_sigmoid1 = StepModel(prefix="sigmoid1_", form="logistic")

        params_sigmoid1 = modelo_sigmoid1.make_params()

        params_sigmoid1.pretty_print(columns=['value', 'min', 'max', 'vary', 'expr'])

        ## Pergunta de parâmetros
        print("Observando o gráfico anterior...")
        print("Qual maior valor pra amplitude da sigmoide? (exemplo: 100)")
        A_max = int(input())
        print("Qual menor valor pra amplitude da sigmoide? (exemplo: 98)")
        A_min = int(input())
        print("Qual valor estimado você atribuiria pra amplitude da sigmoide? (exemplo: 100)")
        A_estimado = int(input())
        print("Qual maior valor para o centro da sigmoide? (exemplo: 30)")
        C_max = int(input())
        print("Qual menor valor para o centro da sigmoide? (exemplo: 20)")
        C_min = int(input())
        print("Qual valor estimado você atribuiria para o centro da sigmoide? (exemplo: 24)")
        C_estimado = int(input())

        ## Redefinição dos parâmetros
        params_sigmoid1["sigmoid1_amplitude"].set(value=A_estimado, min=A_min, max=A_max)
        params_sigmoid1["sigmoid1_center"].set(value=C_estimado, min=C_min, max=C_max)
        params_sigmoid1["sigmoid1_sigma"].set(value=10, min=0, max=100)

        ## Plotando modelo fitado      
        dados_x = df_poco_plotar["Reading"].values
        dados_y = df_poco_plotar["Fluorescence_normalized"].values

        print("Modelagem sigmóide:")

        resultado_fit = modelo_sigmoid1.fit(dados_y, params_sigmoid1, x=dados_x)
        print(resultado_fit.fit_report())

        resultado_fit.plot()
        plt.title(POCO_PLOTAR_i)
        plt.show()

        ## Recolhendo informações do modelo
        print("Copie e cole para as respostas abaixo os valores mostrados nas variáveis na modelagem")
        print("Qual o valor atribuido para a amplitude? (exemplo: 100.000000)")
        A = float(input())
        print("Qual o valor atribuido para o  centro? (exemplo: 23.1398610)")
        u = float(input())
        print("Qual o valor atribuido para o sigma? (exemplo: 2.66142358)")
        o = float(input())
        print("Escreva, dentre as seguintes opções, o que achou da modelagem: Excelente, Muito bom, Razoável/Aceitável, Ruim, Péssimo")
        obs_model = input()
        
        while obs_model.lower() not in ["excelente", "muito bom", "razoável/aceitável", "ruim", "péssimo"]:
            print("Escreva conforme indicado")
            obs_model = input()

        ## Estabelecendo dados obtidos com a sigmoide modelada
        valores_x = range(CORTE_c, CORTE_t+1)
        sigmoid_y = []
        for x in valores_x:
            y = sigmoide(x, A, u, o)
            sigmoid_y.append(y)
        ## Buscando o ponto de inflexão
        medias, derivadas = derivada_xy(valores_x, sigmoid_y)
        p_i = encontra_maximo(derivadas)
        index_p_i = derivadas.index(p_i)
        print("O ponto de inflexão é na máxima derivada igual a ", p_i)
        print("A leitura correspondente a essa derivada é a ", valores_x[index_p_i])
        print("Sua posição no dataframe do poço é ", index_p_i)

        ## Relacionando à Tm
        Tm_i_c = getIndexes(df_poco_plotar, valores_x[index_p_i])
        Tm_i = Tm_i_c[0][0]
        Tm = df_poco_plotar["Temperature"][Tm_i]
        Tms.append(Tm)

        resposta = print("A temperatura de 'melting' é ", Tm)
        
        ## Armazenando valores para plot conjunto
        CORTE_C.append(CORTE_c)
        CORTE_T.append(CORTE_t)
        A_MIN.append(A_min)
        A_MAX.append(A_max)
        A_ESTIMADO.append(A_estimado)
        C_MIN.append(C_min)
        C_MAX.append(C_max)
        C_ESTIMADO.append(C_estimado)
        OBS_MODEL.append(obs_model)
        AMPLITUDE.append(A)
        CENTRO.append(u)
        SIGMA.append(o)
        PONTO_INFLEXAO.append(p_i)
        LEITURA_PI.append(valores_x[index_p_i])
        POSICAO_PI.append(index_p_i)

        return resposta
    
def subplot(poco):
    
    """ Código para configurar subplots dos dados experimentais em uma mesma imagem. """
    
    POCO_PLOTAR_CONJ = pocos.index(poco)
    df_poco_plotar_conj = dfs[POCO_PLOTAR_CONJ]
    return plt.subplot(8, 12, POCO_PLOTAR_CONJ+1), plt.plot(df_poco_plotar_conj["Reading"], df_poco_plotar_conj["Fluorescence"], linestyle="", marker="o"), plt.title(poco)

def subplot_model(poco):
    
    """ Código para configurar subplots dos dados da modelagem em uma mesma imagem, além de incluir classificação e valor de Tm. """
    
    POCO_PLOTAR_CONJ_M = pocos.index(poco)
    df_poco_plotar_conj_m = dfs[POCO_PLOTAR_CONJ_M]
    shape = df_poco_plotar_conj_m.shape

    if shape[1] == 5:
        del df_poco_plotar_conj_m["Fluorescence_normalized"]
        
    
    if OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "desnaturou":
        
        ## Normalização
        eixo_y = list(df_poco_plotar_conj_m["Fluorescence"])
        valor_max = encontra_maximo(eixo_y)
        valor_min = encontra_minimo(eixo_y)
        Fluorescence_normalized = []
        for n in eixo_y:
            novo_valor = (n - valor_min)/(valor_max - valor_min) * 100
            Fluorescence_normalized.append(novo_valor)
        df_poco_plotar_conj_m.insert(4, "Fluorescence_normalized", Fluorescence_normalized, allow_duplicates=False)
        
        return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(df_poco_plotar_conj_m["Reading"], df_poco_plotar_conj_m["Fluorescence_normalized"], linestyle="", marker="o"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="midnightblue")
    
    elif OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() in ["péssimo", "ruim", "razoável/aceitável", "muito bom", "excelente"]:
    
        ## Corte realizado
        logic_c = df_poco_plotar_conj_m["Reading"] >= df_inputs["CORTE_c"][POCO_PLOTAR_CONJ_M]
        df_poco_plotar_conj_m = df_poco_plotar_conj_m.loc[logic_c]
        logic_t = df_poco_plotar_conj_m["Reading"] <= df_inputs["CORTE_t"][POCO_PLOTAR_CONJ_M]
        df_poco_plotar_conj_m = df_poco_plotar_conj_m.loc[logic_t]

        ## Normalização
        eixo_y = list(df_poco_plotar_conj_m["Fluorescence"])
        valor_max = encontra_maximo(eixo_y)
        valor_min = encontra_minimo(eixo_y)
        Fluorescence_normalized = []
        for n in eixo_y:
            novo_valor = (n - valor_min)/(valor_max - valor_min) * 100
            Fluorescence_normalized.append(novo_valor)
        df_poco_plotar_conj_m.insert(4, "Fluorescence_normalized", Fluorescence_normalized, allow_duplicates=False)

        ## Implementação do modelo
        modelo_sigmoid1 = StepModel(prefix="sigmoid1_", form="logistic")

        params_sigmoid1 = modelo_sigmoid1.make_params()

        ## Redefinição dos parâmetros
        params_sigmoid1["sigmoid1_amplitude"].set(value=df_inputs["A_estimado"][POCO_PLOTAR_CONJ_M], min=df_inputs["A_min"][POCO_PLOTAR_CONJ_M], max=df_inputs["A_max"][POCO_PLOTAR_CONJ_M])
        params_sigmoid1["sigmoid1_center"].set(value=df_inputs["C_estimado"][POCO_PLOTAR_CONJ_M], min=df_inputs["C_min"][POCO_PLOTAR_CONJ_M], max=df_inputs["C_max"][POCO_PLOTAR_CONJ_M])
        params_sigmoid1["sigmoid1_sigma"].set(value=10, min=0, max=100)

        ## Plotando modelo fitado      
        dados_x = df_poco_plotar_conj_m["Reading"].values
        dados_y = df_poco_plotar_conj_m["Fluorescence_normalized"].values

        resultado_fit = modelo_sigmoid1.fit(dados_y, params_sigmoid1, x=dados_x)

        if OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "péssimo":
            return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(dados_x, dados_y, 'o'), plt.plot(dados_x, resultado_fit.best_fit, label='best fit', color="maroon"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="maroon")

        elif OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "ruim":
            return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(dados_x, dados_y, 'o'), plt.plot(dados_x, resultado_fit.best_fit, label='best fit', color="red"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="red")

        elif OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "razoável/aceitável":
            return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(dados_x, dados_y, 'o'), plt.plot(dados_x, resultado_fit.best_fit, label='best fit', color="yellow"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="yellow")

        elif OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "muito bom":
            return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(dados_x, dados_y, 'o'), plt.plot(dados_x, resultado_fit.best_fit, label='best fit', color="limegreen"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="limegreen")

        elif OBS_MODEL[POCO_PLOTAR_CONJ_M].lower() == "excelente":
            return plt.subplot(8, 12, POCO_PLOTAR_CONJ_M+1), plt.plot(dados_x, dados_y, 'o'), plt.plot(dados_x, resultado_fit.best_fit, label='best fit', color="darkgreen"), plt.title(poco + " - Tm: " + str(Tms[POCO_PLOTAR_CONJ_M]), color="darkgreen")

**Parte inicial - Definindo base para a análise**

(Só é preciso rodar no começo)

In [ ]:
print("Coloque o nome do arquivo da forma que estiver salvo no computador: (exemplo: arquivo.xls)")
NOME_ARQUIVO = input()
print("Coloque o nome da planilha que quer usar: (exemplo: Melt Curve Raw Data)")
NOME_DA_PLANILHA = input()
print("Coloque, se tiver, em que linha que o cabeçalho termina, sendo que a 1ª é considerada 0: (exemplo: no caso em que o cabeçalho termina na linha 35 da planilha, a resposta deve ser: 34)")
LINHA_CABECALHO = int(input())
print("Coloque quais colunas da tabela contém os dados que quer usar, sendo a 1ª considerada como coluna 0: (exemplo: quer se usar da 2ª até a 5ª coluna, a resposta esperada é: 1, 2, 3, 4)")
COLUNAS_CONTENDO_OS_DADOS_i = input().split(',')
COLUNAS_CONTENDO_OS_DADOS = []
for n in COLUNAS_CONTENDO_OS_DADOS_i:
    COLUNAS_CONTENDO_OS_DADOS.append(int(n))
    
####

df = pd.read_excel(
    NOME_ARQUIVO,
    sheet_name=NOME_DA_PLANILHA,
    header=LINHA_CABECALHO,
    usecols=COLUNAS_CONTENDO_OS_DADOS,
)

df = df.dropna()

####

df.rename(columns={'Well Position': 'Well_Position'}, inplace = True)

####

print("Dataframe de todos os dados a serem usados:")
display(df)

####

groups = df.groupby(df.Well_Position)

####

pocos_r = list(df.Well_Position)
pocos = list(dict.fromkeys(pocos_r))
dfs =[]
for p in pocos:
    poco_df = groups.get_group(p)
    dfs.append(poco_df)
    
####

#dfs

**Tratamento de dados do poço**

* Repita a função:

    tratamento_dados_poco()

    para todo poco que quiser realizar o tratamento de dados. (Não precisa dar argumentos)

* Priorize realizar em ordem, pois os dados serão armazenados na sequência que rodar a função, para montar um heat map.

* É uma análise simples, basta ir respondendo às perguntas que forem sendo mostradas. No decorrer, serão mostrados os Dataframes formados, os gráficos que forem sendo obtidos, a modelagem, variáveis utilizadas e valores obtidos.

In [ ]:
tratamento_dados_poco()

**Heatmap**

Apenas é preciso que rode as células abaixo e responda perguntas, quando aparecerem.

In [ ]:
df_heatmap = pd.DataFrame(pocos)
df_heatmap.insert(1, "Tms", Tms, allow_duplicates=False)

colunas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
linhas = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D','D', 'E', 'E', 'E', 'E','E', 'E', 'E', 'E','E','E','E', 'E', 'F', 'F', 'F','F', 'F','F', 'F','F','F','F','F','F', 'G', 'G','G','G','G','G','G','G','G','G','G','G', 'H', 'H', 'H','H', 'H', 'H', 'H', 'H','H','H','H','H']
df_heatmap.insert(1, "Colunas", colunas, allow_duplicates=False)
df_heatmap.insert(1, "Linhas", linhas, allow_duplicates=False)

del df_heatmap[0]

df_heatmap

In [ ]:
df_heatmap = df_heatmap.pivot("Linhas", "Colunas", "Tms")
df_heatmap

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.heatmap(df_heatmap, cmap="seismic", annot=True, fmt='.2f')
ax.figure.savefig("heatmap_original", dpi=300)

Essa imagem ficará salva na mesma pasta deste arquivo com o nome "heatmap_original.png" em alta resolução.

In [ ]:
print("Qual o menor valor, exceto 0, que é possível observar? (exemplo: 33.91)")
VMIN = float(input())
print("Qual o maior valor que é possível observar? (exemplo: 44.83)")
VMAX = float(input())
print("Quantas casas depois da vírgula quer que apareça? (use da seguinte forma: quer que apareça 2 casas, resposta: .2f)")
FMT = input()

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.heatmap(df_heatmap, vmin = VMIN, vmax = VMAX, cmap="seismic", annot=True, fmt=FMT)
ax.figure.savefig("heatmap_escala", dpi=300)

Essa imagem ficará salva na mesma pasta deste arquivo com o nome "heatmap_escala.png" em alta resolução.

**Gráficos**

Apenas é preciso que rode as células abaixo.

A plotagem foi esquematizada considerando o uso de uma plaquinha com 8 linhas e 12 colunas, ou seja, de 96 poços.

- Dados experimentais

In [ ]:
plt.style.use("seaborn")
%matplotlib inline
plt.figure(figsize = ((40,24)))
plt.rcParams.update({'figure.max_open_warning': 0})

subplot("A1")
subplot("A2")
subplot("A3")
subplot("A4")
subplot("A5")
subplot("A6")
subplot("A7")
subplot("A8")
subplot("A9")
subplot("A10")
subplot("A11")
subplot("A12")

subplot("B1")
subplot("B2")
subplot("B3")
subplot("B4")
subplot("B5")
subplot("B6")
subplot("B7")
subplot("B8")
subplot("B9")
subplot("B10")
subplot("B11")
subplot("B12")

subplot("C1")
subplot("C2")
subplot("C3")
subplot("C4")
subplot("C5")
subplot("C6")
subplot("C7")
subplot("C8")
subplot("C9")
subplot("C10")
subplot("C11")
subplot("C12")

subplot("D1")
subplot("D2")
subplot("D3")
subplot("D4")
subplot("D5")
subplot("D6")
subplot("D7")
subplot("D8")
subplot("D9")
subplot("D10")
subplot("D11")
subplot("D12")

subplot("E1")
subplot("E2")
subplot("E3")
subplot("E4")
subplot("E5")
subplot("E6")
subplot("E7")
subplot("E8")
subplot("E9")
subplot("E10")
subplot("E11")
subplot("E12")

subplot("F1")
subplot("F2")
subplot("F3")
subplot("F4")
subplot("F5")
subplot("F6")
subplot("F7")
subplot("F8")
subplot("F9")
subplot("F10")
subplot("F11")
subplot("F12")

subplot("G1")
subplot("G2")
subplot("G3")
subplot("G4")
subplot("G5")
subplot("G6")
subplot("G7")
subplot("G8")
subplot("G9")
subplot("G10")
subplot("G11")
subplot("G12")

subplot("H1")
subplot("H2")
subplot("H3")
subplot("H4")
subplot("H5")
subplot("H6")
subplot("H7")
subplot("H8")
subplot("H9")
subplot("H10")
subplot("H11")
subplot("H12")

plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.35)
plt.suptitle("Gráficos - Dados Experimentais", fontsize = 20)

plt.savefig ("gráficos_experimental.png", dpi=300)

plt.show()

Essa imagem ficará salva na mesma pasta deste arquivo com o nome "gráficos_experimental.png" em alta resolução.

- Dados cortados, modelagem, classificação e Tm

In [ ]:
tuplas_inputs = list(zip(CORTE_C, CORTE_T, A_MIN, A_MAX, A_ESTIMADO, C_MIN, C_MAX, C_ESTIMADO)) 

df_inputs = pd.DataFrame(
    tuplas_inputs,
    columns=['CORTE_c', 'CORTE_t', 'A_min', 'A_max', 'A_estimado', 'C_min', 'C_max', 'C_estimado']
)

display(df_inputs)

Colorcode da classificação da modelagem:

**<p style="color:darkgreen;"> Excelente</p>**
**<p style="color:limegreen;"> Muito bom</p>**
**<p style="color:yellow;"> Razoável/Aceitável</p>**
**<p style="color:red;"> Ruim</p>**
**<p style="color:maroon;"> Péssimo</p>**
**<p style="color:midnightblue;"> Desnaturou</p>**

In [ ]:
%matplotlib inline
plt.figure(figsize = ((40,24)))
plt.rcParams.update({'figure.max_open_warning': 0})

subplot_model("A1")
subplot_model("A2")
subplot_model("A3")
subplot_model("A4")
subplot_model("A5")
subplot_model("A6")
subplot_model("A7")
subplot_model("A8")
subplot_model("A9")
subplot_model("A10")
subplot_model("A11")
subplot_model("A12")

subplot_model("B1")
subplot_model("B2")
subplot_model("B3")
subplot_model("B4")
subplot_model("B5")
subplot_model("B6")
subplot_model("B7")
subplot_model("B8")
subplot_model("B9")
subplot_model("B10")
subplot_model("B11")
subplot_model("B12")

subplot_model("C1")
subplot_model("C2")
subplot_model("C3")
subplot_model("C4")
subplot_model("C5")
subplot_model("C6")
subplot_model("C7")
subplot_model("C8")
subplot_model("C9")
subplot_model("C10")
subplot_model("C11")
subplot_model("C12")

subplot_model("D1")
subplot_model("D2")
subplot_model("D3")
subplot_model("D4")
subplot_model("D5")
subplot_model("D6")
subplot_model("D7")
subplot_model("D8")
subplot_model("D9")
subplot_model("D10")
subplot_model("D11")
subplot_model("D12")

subplot_model("E1")
subplot_model("E2")
subplot_model("E3")
subplot_model("E4")
subplot_model("E5")
subplot_model("E6")
subplot_model("E7")
subplot_model("E8")
subplot_model("E9")
subplot_model("E10")
subplot_model("E11")
subplot_model("E12")

subplot_model("F1")
subplot_model("F2")
subplot_model("F3")
subplot_model("F4")
subplot_model("F5")
subplot_model("F6")
subplot_model("F7")
subplot_model("F8")
subplot_model("F9")
subplot_model("F10")
subplot_model("F11")
subplot_model("F12")

subplot_model("G1")
subplot_model("G2")
subplot_model("G3")
subplot_model("G4")
subplot_model("G5")
subplot_model("G6")
subplot_model("G7")
subplot_model("G8")
subplot_model("G9")
subplot_model("G10")
subplot_model("G11")
subplot_model("G12")

subplot_model("H1")
subplot_model("H2")
subplot_model("H3")
subplot_model("H4")
subplot_model("H5")
subplot_model("H6")
subplot_model("H7")
subplot_model("H8")
subplot_model("H9")
subplot_model("H10")
subplot_model("H11")
subplot_model("H12")

plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.35)
plt.suptitle("Gráficos - Dados da Modelagem", fontsize = 20)

plt.savefig ("gráficos_modelagem.png", dpi=300)

plt.show()

Essa imagem ficará salva na mesma pasta deste arquivo com o nome "gráficos_modelagem.png" em alta resolução.